# ¿Cómo medir rendimiento y riesgo en un portafolio? II

<img style="float: right; margin: 0px 0px 15px 15px;" src="http://www.picpedia.org/clipboard/images/stock-portfolio.jpg" width="600px" height="400px" />

> La clase pasada y la presente, están dedicadas a obtener medidas de rendimiento y riesgo en un portafolio.

> Vimos que podemos obtener los rendimientos de un portafolio mediante la relación $r_p=\sum_{i=1}^{n}w_ir_i$, y una vez teniendo los rendimientos del portafolio, lo podemos tratar como un activo individual.

> Por otra parte, vimos que si conocemos los rendimientos esperados de cada activo que conforma el portafolio $E[r_i]$, podemos calcular el rendimiento esperado del portafolio como el promedio ponderado de los rendimientos esperados de los activos $E[r_p]=\sum_{i=1}^{n}w_iE[r_i]$.

> Sin embargo, vimos que esto no es válido para la medida de riesgo (desviación estándar). Es decir, la varianza (o volatilidad, o desviación estándar) no es el promedio ponderado de las varianzas individuales. Anticipamos que esto es clave en el concepto de **diversificación**.

**Objetivos:**
- Medir el riesgo en un portafolio a partir del riesgo de cada uno de los activos que lo conforman.

*Referencias*
- Notas del curso "Portfolio Selection and Risk Management", Rice University, disponible en Coursera.
___

## 1. Midiendo el riesgo en un portafolio

### 1.1. Volatilidad de un portafolio

Retomamos el ejemplo qur veníamos trabajando la clase pasada...

**Ejemplo.** Supongamos que tenemos inversión en activos de Toyota, Walmart y Pfizer. Tenemos cuatro posibles estados económicos:

In [1]:
import numpy as np
import pandas as pd

In [3]:
# Creamos tabla
tabla = pd.DataFrame(columns=['Prob', 'Toyota', 'Walmart', 'Pfizer'], index=['Expansion', 'Normal', 'Recesion'])
tabla.index.name = 'Estado'
tabla['Prob']=np.array([0.3, 0.4, 0.3])
tabla['Toyota']=np.array([-0.2, 0.05, 0.4])
tabla['Walmart']=np.array([-0.05, 0.1, 0.15])
tabla['Pfizer']=np.array([0.05, 0.03, 0.02])

tabla.round(4)

Prob  Toyota  Walmart  Pfizer
Estado                                  
Expansion   0.3   -0.20    -0.05    0.05
Normal      0.4    0.05     0.10    0.03
Recesion    0.3    0.40     0.15    0.02

In [5]:
# Incluimos una fila para rendimientos esperados
ErT=(tabla['Prob']*tabla['Toyota']).sum()
ErW=(tabla['Prob']*tabla['Walmart']).sum()
ErP=(tabla['Prob']*tabla['Pfizer']).sum()
tabla.loc['Media',:]=[None,
                      ErT,
                      ErW,
                      ErP]
tabla.round(4)

Prob  Toyota  Walmart  Pfizer
Estado                                  
Expansion   0.3   -0.20    -0.05   0.050
Normal      0.4    0.05     0.10   0.030
Recesion    0.3    0.40     0.15   0.020
Media       NaN    0.08     0.07   0.033

In [15]:
## Volatilidad
# Toyota
#sT = ((tabla['Prob']*(tabla['Toyota']*ErT)**2).sum())**0.5
# Walmart
#sW = ((tabla['Prob']*(tabla['Walmart']*ErW)**2).sum())**0.5
# Pfizer
#sP = ((tabla['Prob']*(tabla['Pfizer']*ErP)**2).sum())**0.5
# Mostrar
sT=0.2337
sW=0.0812
sP=0.0119

In [24]:
# Portafolio 0.5Toyota+0.5Pfizer
tabla['Ptyp'] = 0.5*tabla['Toyota']+0.5*tabla['Pfizer']
tabla

Prob  Toyota  Walmart  Pfizer    Ptyp
Estado                                          
Expansion   0.3   -0.20    -0.05   0.050 -0.0750
Normal      0.4    0.05     0.10   0.030  0.0400
Recesion    0.3    0.40     0.15   0.020  0.2100
Media       NaN    0.08     0.07   0.033  0.0565

In [26]:
# Volatilidad del portafolio
ErPtyp=tabla.iloc[-1,-1]
stPtyp=((tabla['Prob']*(tabla['Ptyp']*ErPtyp)**2).sum())**0.5
stPtyp

0.007047228488916761

**Actividad.** Encontrar la volatilidad del portafolio formado $0.5$ Toyota y $0.5$ Walmart.

In [7]:
# Encontrar los rendimientos del portafolio en cada estado de la economía


In [21]:
# Encontrar el rendimiento esperado del portafolio
tabla['Ptyw'] = 0.5*tabla['Toyota']+0.5*tabla['Walmart']
tabla

Prob  Toyota  Walmart  Pfizer     Ptyp     Ptyw
Estado                                                    
Expansion   0.1   0.060   0.0450  0.0250  0.04250  0.05250
Normal      0.4   0.075   0.0550 -0.0050  0.03500  0.06500
Recesion    0.3   0.020   0.0400  0.0100  0.01500  0.03000
Depresion   0.2  -0.030  -0.0100  0.1300  0.05000 -0.02000
Media       NaN   0.036   0.0365  0.0295  0.03275  0.03625

In [22]:
ErPtyw=tabla.iloc[-1,-1]
ErPtyw

0.036250000000000004

In [20]:
# Encontrar la volatilidad de Toyota, Walmart y el portafolio
stPtyw=((tabla['Prob']*(tabla['Ptyw']*ErPtyw)**2).sum())**0.5
stPtyw

0.001575960830434088

### 1.2. Midiendo el co-movimiento entre instrumentos

- Una vez más, concluimos que la volatilidad (varianza) **NO** es el promedio ponderado de las varianzas individales.

- Por el contrario, la varianza de los rendimientos de un portafolio está afectada por el movimiento relativo de un activo individual respecto a otro.

- Por tanto, necesitamos definir las medidas de **covarianza** y **correlación**, que nos permiten evaluar las fluctuaciones relativas entre los activos.

#### Covarianza:

Es una medida el movimiento relativo entre dos instrumentos.

Matemáticamente, si tenemos dos activos $A_1$ y $A_2$ cuyos rendimientos son $r_1$ y $r_2$, respectivamente, entonces la covarianza de los rendimientos de los activos es

$$\text{cov}(r_1,r_2)=\sigma_{12}=\sigma_{21}=\sum_{j=1}^{m}p_j(r_{1j}-E[r_1])(r_{2j}-E[r_2]).$$

Podemos notar fácilmente que la covarianza de los rendimientos de un activo con los rendimientos del mismo activo corresponde a la varianza

$$\text{cov}(r_1,r_1)=\sigma_{11}=\sum_{j=1}^{m} p_j(r_{1j}-E[r_1])(r_{1j}-E[r_1])=\sigma_1^2=\text{var}(r_1).$$

**Ejemplo.** Calcular la covarianza entre los rendimientos de Toyota y Pfizer.

In [23]:
# Mostrar tabla
tabla

Prob  Toyota  Walmart  Pfizer     Ptyp     Ptyw
Estado                                                    
Expansion   0.1   0.060   0.0450  0.0250  0.04250  0.05250
Normal      0.4   0.075   0.0550 -0.0050  0.03500  0.06500
Recesion    0.3   0.020   0.0400  0.0100  0.01500  0.03000
Depresion   0.2  -0.030  -0.0100  0.1300  0.05000 -0.02000
Media       NaN   0.036   0.0365  0.0295  0.03275  0.03625

In [16]:
# Calcular la covarianza
covTP = (tabla['Prob']*((tabla['Toyota']-ErT)*(tabla['Pfizer']-ErP))).sum()
covTP

-0.0026400000000000004

**Actividad.** Calcular la covarianza entre los rendimientos de Toyota y Walmart.

In [20]:
# Calcular la covarianza
covTW = (tabla['Prob']*((tabla['Pfizer']-ErT)*(tabla['Walmart']-ErW))).sum()
covTW

-0.0009599999999999997

¿Qué nos dice este número?
- El signo nos dice las direcciones relativas entre los rendimientos de cada activo. Por ejemplo, la covarianza entre los rendimientos de Toyota y Pfizer es negativa... ver los rendimientos.
- La magnitud de la covarianza no nos dice mucho acerca de la fuerza con que se relacionan o no estos rendimientos.

**Correlación**
Un posible problema de la covarianza es que la magnitud de esta medida no nos dice mucho acerca de la fuerza de los movimientos relativos. La *correlación* es una medida normalizada del movimiento relativo entre los rendimientos de dos activos.

Matemáticamente,

$$\text{corr}(r_1,r_2)=\rho_{12}=\rho_{21}=\frac{\sigma_{12}}{\sigma_1\sigma_{2}}.$$

Propiedades:

- Podemos notar fácilmente que la correlación de los rendimientos de un activo con los rendimientos del mismo activo es $1$: $$\text{corr}(r_1,r_1)=\rho_{11}=\frac{\sigma_{11}}{\sigma_1\sigma_1}=\frac{\sigma_{1}^2}{\sigma_1\sigma_1}=1.$$
- El signo de la correlación y la covarianza es el mismo.
- La correlación satisface: $$-1\leq\rho_{12}\leq 1.$$

**Ejemplo.** Calcular la correlación entre los rendimientos de Toyota y Pfizer.

In [18]:
corTP=covTP/(sT*sP)
corTP

-0.9492885729388033

**Actividad.** Calcular la correlación entre los rendimientos de Toyota y Walmart.

In [21]:
corTW=covTW/(sP*sW)
corTW

-0.9935008486153079

**Conclusión.**
- Es una medida normalizada de la fluctuación relativa de los rendimientos de dos activos.
- En los ejemplos que vimos, sería conveniente invertir en el portafolio de Toyota y Pfizer puesto que su correlación es negativa, y esto impactaría positivamente en la diversificación del riesgo.

___

## 2. Uniendo todo...

- Entonces, vimos mediante ejemplos que el riesgo en un portafolio se ve afectado significativamente por como los rendimientos de los activos se mueven relativamente.
- Este movimiento relativo lo medimos mediante la covarianza o la correlación.
- Si se mueven de una manera que no están perfectamente correlacionados ($\rho<1$), entonces el riesgo del portafolio siempre será menor que el promedio ponderado de los riesgos individuales.

<img style="float: left; margin: 0px 0px 15px 15px;" src="https://www.publicdomainpictures.net/pictures/20000/velka/happy-child.jpg" width="300px" height="200px" />

## Ésta es la razón por la que combinar activos en un portafolio permite diversificar el riesgo...

Entonces, ¿cómo podemos incorporar esta medida en el cálculo de la varianza del portafolio? 
- <font color=blue> Ver en el tablero...</font>
- ¿Cómo sería para dos activos?

**Ejemplo.** Calcular por fórmula para el portafolio de Toyota y Pfizer. Comparar.

In [35]:
stPtyp

0.0011523742644264927

In [ ]:
covPTP = np.sqrt((0.5**2*sT**2+0.5**2*sP**2+20.5**2*0.5*covTP))

**Actividad.** Calcular por fórmula para el portafolio de Toyota y Walmart. Comparar.

## 2.1. <font color=blue> Ver en el tablero...</font>
### Matriz de varianza covarianza.
### Matriz de correlación.

# Anuncios parroquiales

## 1. Recordar quiz la siguiente clase. Temas: Clases 7 y 8.
## 2. Tarea: revisar archivo "Tarea4_MidiendoRendimientoRiesgo" en clase. Para ...
## 3. Segunda entrega Tarea 3 para ...

<script>
  $(document).ready(function(){
    $('div.prompt').hide();
    $('div.back-to-top').hide();
    $('nav#menubar').hide();
    $('.breadcrumb').hide();
    $('.hidden-print').hide();
  });
</script>

<footer id="attribution" style="float:right; color:#808080; background:#fff;">
Created with Jupyter by Esteban Jiménez Rodríguez.
</footer>